In [ ]:
import napari
import numpy as np
from pathlib import Path

import sys

from tqdm.notebook import tqdm

from faim_ipa.utils import get_git_root, make_relative_to_git_root

from glob import glob
from os.path import basename

root = get_git_root()
sys.path.append(str(root / 'source' / 's03_compute_tile_masks'))

from tile_masking_section import TileMaskingSection

# Welcome to Proof Reading

This notebook uses napari to visualize tiles and masks for a selected section. The masks can be manually edited and resaved if necessary.

## Create Viewer
First we will open the napari viewer. After executing the next cell the viewer will open, but no data will be displayed.

In [ ]:
viewer = napari.Viewer()

## Select Section
Now you need to provide the path to a section directory. Go to your file-browser and navigate to the section (e.g. 's1012_g0') you would like to inspect and copy paste the path to the dirctory into the next line.

In [ ]:
section_directory = Path('put-your-path-here')

In [ ]:
section = TileMaskingSection.load_from_yaml(section_directory / 'section.yaml')

The next code-cell will add all tiles and tile-masks to the napari viewer. Once it has completed you can switch to the napari viewer and inspect them. You can also edit the masks and save them later.

In [ ]:
shape = section.get_tile_shape()
tile_id_map = section.get_tile_id_map(path=section_directory / 'tile_id_map.json')

changed_masks = {}
mask_layers = {}

def set_masked_changed(tile_id):
    def masked_changed():
        changed_masks[tile_id] = True
    return masked_changed

for tile_id, tile in tqdm(section.tiles.items()):
    try:
        y, x = np.where(tile_id_map == tile_id)
        # Make sure that tile_id is part of the tile_id_map
        y = y[0]
        x = x[0]
        
        viewer.add_image(tile.get_tile_data(), 
                        translate=(y * shape[0], x * shape[1] ), name=f"tile-{tile_id} ({y}, {x})")
        mask = np.zeros(shape, dtype=np.uint8)
        mask[section.get_smearing_mask(tile_id)] = 1
        mask[section.get_resin_mask(tile_id)] = 2
        mask_layer = viewer.add_labels(mask, 
                        translate=(y * shape[0], x * shape[1] ), name=f"mask-{tile_id} ({y}, {x})")
        changed_masks[tile_id] = False
        mask_layer.events.paint.connect(set_masked_changed(tile_id))
        mask_layers[tile_id] = mask_layer
    except IndexError as e:
        print(f"{tile_id} is not part of tile_id_map")

## Save Edited Masks
Running the next cell will overwrite the existing masks with your edited masks from the viewer.

To avoid overwriting them by accident you are required to comment the top-most line (`assert ...`).

In [ ]:
assert False, 'Comment the top line, if you want to save and overwrite the existing masks.'
for tile_id, has_changed in changed_masks.items():
    if has_changed:
        masks = mask_layers[tile_id].data
        section.set_smearing_mask(tile_id, masks == 1)
        print(f'Updated masks for {tile_id}.')

section.save(str(sections_dir.parent), overwrite=False)